In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
#import pymc3 as pm
#import bambi as bmb
import mne

In [5]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 3.9 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [41]:
data = pd.read_csv('results/data_beh_eeg.csv',sep='\t')
data.dropna(subset=['rt','cpp_amplitude','cpp_peak','cpp_slopes'], inplace=True)
data['cpp_slopes']=StandardScaler().fit_transform(data['cpp_slopes'].to_numpy().reshape(-1,1))

data['cpp_amplitude']=StandardScaler().fit_transform(data['cpp_amplitude'].to_numpy().reshape(-1,1))

data['cpp_peak']=StandardScaler().fit_transform(data['cpp_peak'].to_numpy().reshape(-1,1))

In [48]:
data

,condition,coherence,prioritized,subj_idx,rt,key_press,correct,run,stim,n170_amplitude,n170_slopes,baseline,cpp_amplitude,cpp_slopes,cpp_var,cpp_peak,cpp_peak_latency
0,1.0,high,yes,sub-001,0.529206,face,1.0,1,face,-0.000006,2.340534e-07,-7.629888e-22,0.648190,2.001119,1.560474e-11,0.871625,0.417096
1,2.0,high,no,sub-001,0.519501,car,1.0,1,car,NaN,NaN,9.063582e-24,1.225435,-0.174493,7.937925e-12,1.264771,0.346845
2,2.0,high,no,sub-001,0.611204,face,1.0,1,face,0.000002,-9.800590e-07,-5.932526e-23,0.160142,0.312096,8.048211e-12,0.486489,0.489329
3,1.0,high,yes,sub-001,0.465089,car,1.0,1,car,NaN,NaN,2.175260e-22,-1.837984,0.491916,2.988462e-12,-1.845775,0.360793
4,1.0,high,yes,sub-001,0.367166,car,1.0,1,car,NaN,NaN,-6.591696e-23,-1.333245,1.411133,1.284015e-11,-1.297068,0.262869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,3.0,low,yes,sub-017,0.309091,car,1.0,2,car,NaN,NaN,4.268123e-22,0.095669,0.163095,7.588991e-12,0.036959,0.161825
4604,3.0,low,yes,sub-017,0.261170,car,1.0,2,car,NaN,NaN,-2.389490e-23,0.654608,-0.030639,9.850273e-13,-0.179208,0.088514
4605,3.0,low,yes,sub-017,0.271483,car,0.0,2,face,-0.000009,1.896555e-07,5.586462e-22,-0.146989,-0.062619,7.014867e-12,-0.269449,0.092968
4606,4.0,low,no,sub-017,0.283706,car,1.0,2,car,NaN,NaN,-1.977509e-22,-0.003302,-0.105004,1.355688e-12,-0.487029,0.154019


In [29]:
X = StandardScaler().fit_transform(data.groupby('subj_idx').mean().cpp_slopes.values.reshape(-1, 1))
y = data.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     17.51
Date:                Mon, 27 Mar 2023   Prob (F-statistic):           0.000918
Time:                        14:01:28   Log-Likelihood:                 20.668
No. Observations:                  16   AIC:                            -37.34
Df Residuals:                      14   BIC:                            -35.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4458      0.018     25.087      0.0

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [66]:
model  = bmb.Model('rt~1+cpp_slopes+coherence+cpp_slopes:coherence+(1|subj_idx)',data)
results = model.fit(draws=2000,tune=1000,chains=4)
az.summary(results)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [rt_sigma, 1|subj_idx_offset, 1|subj_idx_sigma, cpp_slopes:coherence, coherence, cpp_slopes, Intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 19 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.404,0.029,0.351,0.463,0.001,0.001,726.0,611.0,1.0
cpp_slopes,-0.004,0.004,-0.011,0.004,0.000,0.000,4316.0,4545.0,1.0
coherence[low],0.085,0.006,0.074,0.096,0.000,0.000,5666.0,3800.0,1.0
cpp_slopes:coherence[low],0.001,0.006,-0.009,0.012,0.000,0.000,4339.0,3840.0,1.0
1|subj_idx_sigma,0.111,0.023,0.072,0.154,0.001,0.001,946.0,880.0,1.0
1|subj_idx[sub-001],0.012,0.031,-0.050,0.069,0.001,0.001,819.0,633.0,1.0
1|subj_idx[sub-003],0.016,0.031,-0.043,0.073,0.001,0.001,787.0,622.0,1.0
1|subj_idx[sub-004],0.026,0.031,-0.035,0.083,0.001,0.001,768.0,629.0,1.0
1|subj_idx[sub-005],-0.053,0.031,-0.112,0.006,0.001,0.001,784.0,629.0,1.0
1|subj_idx[sub-006],0.099,0.031,0.037,0.153,0.001,0.001,765.0,660.0,1.0


In [30]:
X = StandardScaler().fit_transform(data.groupby('subj_idx').mean().cpp_amplitude.values.reshape(-1, 1))
y = data.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.071
Method:                 Least Squares   F-statistic:                   0.01157
Date:                Mon, 27 Mar 2023   Prob (F-statistic):              0.916
Time:                        14:01:29   Log-Likelihood:                 14.184
No. Observations:                  16   AIC:                            -24.37
Df Residuals:                      14   BIC:                            -22.82
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4458      0.027     16.729      0.0

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [rt_sigma, 1|subj_idx_offset, 1|subj_idx_sigma, cpp_amplitude:coherence, coherence, cpp_amplitude, Intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 19 seconds.
There were 19 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.404,0.028,0.351,0.458,0.001,0.001,949.0,1005.0,1.01
cpp_amplitude,-0.002,0.004,-0.009,0.006,0.000,0.000,4146.0,4263.0,1.00
coherence[low],0.085,0.006,0.074,0.095,0.000,0.000,5446.0,4737.0,1.00
cpp_amplitude:coherence[low],-0.007,0.006,-0.017,0.004,0.000,0.000,4819.0,5332.0,1.00
1|subj_idx_sigma,0.111,0.023,0.074,0.157,0.001,0.000,1144.0,2007.0,1.00
1|subj_idx[sub-001],0.010,0.030,-0.048,0.065,0.001,0.001,1060.0,1137.0,1.00
1|subj_idx[sub-003],0.014,0.030,-0.041,0.072,0.001,0.001,1015.0,1120.0,1.01
1|subj_idx[sub-004],0.030,0.030,-0.025,0.088,0.001,0.001,1045.0,1093.0,1.01
1|subj_idx[sub-005],-0.052,0.030,-0.111,0.002,0.001,0.001,1072.0,1042.0,1.01
1|subj_idx[sub-006],0.098,0.030,0.041,0.155,0.001,0.001,1051.0,1033.0,1.01


In [31]:
X = StandardScaler().fit_transform(data.groupby('subj_idx').mean().cpp_peak.values.reshape(-1, 1))
y = data.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.1506
Date:                Mon, 27 Mar 2023   Prob (F-statistic):              0.704
Time:                        14:01:29   Log-Likelihood:                 14.263
No. Observations:                  16   AIC:                            -24.53
Df Residuals:                      14   BIC:                            -22.98
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4458      0.027     16.812      0.0

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [64]:
model  = bmb.Model('rt~1+cpp_peak+(1|subj_idx)',data)
results = model.fit(draws=2000,tune=1000,chains=4)
az.summary(results)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [rt_sigma, cpp_peak|subj_idx_offset, cpp_peak|subj_idx_sigma, 1|subj_idx_offset, 1|subj_idx_sigma, cpp_peak, Intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 35 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.451,0.028,0.399,0.505,0.001,0.001,1480.0,2207.0,1.0
cpp_peak,-0.004,0.009,-0.021,0.013,0.000,0.000,2743.0,3966.0,1.0
1|subj_idx_sigma,0.112,0.023,0.075,0.154,0.001,0.000,1939.0,3053.0,1.0
1|subj_idx[sub-001],0.007,0.030,-0.049,0.065,0.001,0.001,1678.0,2770.0,1.0
1|subj_idx[sub-003],0.008,0.031,-0.050,0.065,0.001,0.001,1715.0,2863.0,1.0
1|subj_idx[sub-004],0.049,0.032,-0.009,0.109,0.001,0.001,1807.0,3242.0,1.0
1|subj_idx[sub-005],-0.055,0.031,-0.110,0.004,0.001,0.001,1678.0,2830.0,1.0
1|subj_idx[sub-006],0.104,0.031,0.044,0.161,0.001,0.001,1788.0,2936.0,1.0
1|subj_idx[sub-007],0.046,0.030,-0.007,0.106,0.001,0.001,1671.0,2682.0,1.0
1|subj_idx[sub-008],-0.065,0.030,-0.119,-0.006,0.001,0.001,1690.0,2952.0,1.0


In [75]:
model  = bmb.Model('rt~1+cpp_peak+coherence+cpp_peak:coherence+(1|subj_idx)',data)
results = model.fit(draws=2000,tune=1000,chains=4)
az.summary(results)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [rt_sigma, 1|subj_idx_offset, 1|subj_idx_sigma, cpp_peak:coherence, coherence, cpp_peak, Intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 19 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 256 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6588154504860904, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,0.401,0.028,0.349,0.453,0.001,0.001,992.0,1083.0,1.01
cpp_peak,-0.002,0.004,-0.010,0.006,0.000,0.000,2229.0,2112.0,1.00
coherence[low],0.085,0.006,0.074,0.095,0.000,0.000,3814.0,3324.0,1.00
cpp_peak:coherence[low],-0.003,0.006,-0.013,0.008,0.000,0.000,2630.0,2392.0,1.00
1|subj_idx_sigma,0.111,0.023,0.072,0.152,0.001,0.001,959.0,1367.0,1.01
1|subj_idx[sub-001],0.014,0.029,-0.041,0.070,0.001,0.001,1042.0,1376.0,1.00
1|subj_idx[sub-003],0.018,0.030,-0.035,0.077,0.001,0.001,1077.0,1316.0,1.01
1|subj_idx[sub-004],0.031,0.030,-0.023,0.088,0.001,0.001,1070.0,1263.0,1.01
1|subj_idx[sub-005],-0.049,0.030,-0.105,0.006,0.001,0.001,1135.0,1612.0,1.01
1|subj_idx[sub-006],0.101,0.030,0.045,0.154,0.001,0.001,1070.0,1083.0,1.01


In [32]:
X = StandardScaler().fit_transform(data.groupby('subj_idx').mean().cpp_peak_latency.values.reshape(-1, 1))
y = data.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     5077.
Date:                Mon, 27 Mar 2023   Prob (F-statistic):           2.49e-19
Time:                        14:01:29   Log-Likelihood:                 61.348
No. Observations:                  16   AIC:                            -118.7
Df Residuals:                      14   BIC:                            -117.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4458      0.001    318.890      0.0

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [33]:
data_n170 = data.dropna()
X = StandardScaler().fit_transform(data_n170.groupby('subj_idx').mean().n170_slopes.values.reshape(-1, 1))
y = data_n170.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     2.254
Date:                Mon, 27 Mar 2023   Prob (F-statistic):              0.156
Time:                        14:01:30   Log-Likelihood:                 16.580
No. Observations:                  16   AIC:                            -29.16
Df Residuals:                      14   BIC:                            -27.62
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4420      0.023     19.264      0.0

In [34]:
X = StandardScaler().fit_transform(data_n170.groupby('subj_idx').mean().n170_amplitude.values.reshape(-1, 1))
y = data_n170.groupby('subj_idx').mean().rt.values
X = sm.add_constant(X)
model = sm.OLS(y, X)

# 拟合模型
results = model.fit()

# 输出结果
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.067
Method:                 Least Squares   F-statistic:                   0.05296
Date:                Mon, 27 Mar 2023   Prob (F-statistic):              0.821
Time:                        14:01:30   Log-Likelihood:                 15.416
No. Observations:                  16   AIC:                            -26.83
Df Residuals:                      14   BIC:                            -25.29
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4420      0.025     17.912      0.0

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [35]:
x1 = data[data['coherence']=='high'].groupby(['subj_idx']).mean().cpp_peak.values.reshape(-1, 1)
x2 = data[data['coherence']=='low'].groupby(['subj_idx']).mean().cpp_peak.values.reshape(-1, 1)
stats.ttest_rel(x1, x2)

Ttest_relResult(statistic=array([1.02185756]), pvalue=array([0.32304708]))

In [36]:
x1 = data[data['prioritized']=='yes'].groupby(['subj_idx']).mean().cpp_peak.values.reshape(-1, 1)
x2 = data[data['prioritized']=='no'].groupby(['subj_idx']).mean().cpp_peak.values.reshape(-1, 1)
stats.ttest_rel(x1, x2)

Ttest_relResult(statistic=array([-1.5350541]), pvalue=array([0.14559148]))

In [37]:
x1 = data[data['coherence']=='high'].groupby(['subj_idx']).mean().cpp_slopes.values.reshape(-1, 1)
x2 = data[data['coherence']=='low'].groupby(['subj_idx']).mean().cpp_slopes.values.reshape(-1, 1)
stats.ttest_rel(x1, x2)

Ttest_relResult(statistic=array([1.42396333]), pvalue=array([0.17492672]))

In [38]:
x1 = data[data['coherence']=='high'].groupby(['subj_idx']).mean().cpp_amplitude.values.reshape(-1, 1)
x2 = data[data['coherence']=='low'].groupby(['subj_idx']).mean().cpp_amplitude.values.reshape(-1, 1)
stats.ttest_rel(x1, x2)

Ttest_relResult(statistic=array([1.86061065]), pvalue=array([0.08251735]))

## permutate_test

In [22]:
def permutate_var(subj, ntimes):
    
    subj_str = f'sub-{subj:03d}'
    raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)




    stimulus_dict = {
    'Stimulus/hc/p/left': 4,
    'Stimulus/hc/p/right': 5,
    'Stimulus/hc/np/left': 6,
    'Stimulus/hc/np/right': 7,
    'Stimulus/lc/p/left': 8,
    'Stimulus/lc/p/right': 9,
    'Stimulus/lc/np/left': 10,
    'Stimulus/lc/np/right': 11,
    }

    response_dict = {
    'Response/car': 2,
    'Response/face': 3,
    }

    stim_dict = {
        'Stimulus/car': 16,
        'Stimulus/face': 17,
    }

    data = pd.read_csv('data_beh_eeg.csv', sep='\t')
    data = data[data['subj_idx']==subj_str]



    # N170
    events_from_annot, event_dict = mne.events_from_annotations(raw, event_id={
        'Stimulus/S 10': 4,
        'Stimulus/S 11': 5,
        'Stimulus/S 20': 6,
        'Stimulus/S 21': 7,
        'Stimulus/S 30': 8,
        'Stimulus/S 31': 9,
        'Stimulus/S 40': 10,
        'Stimulus/S 41': 11,})


    if len(events_from_annot) != len(data):
        new_events = [1 if x == 4 or x == 5 else 2 if x == 6 or x == 7 else 3 if x == 8 or x == 9 else 4 for x in events_from_annot[:,2].tolist()]
        boo = np.zeros(len(data))

        
        diff_len = len(data) - len(new_events)
        new_events += [4] * diff_len
        
        boo = np.array(data['condition'] == new_events, dtype=np.int32)

        loc = []
        for i in range(len(boo)):
            boo = np.array(data['condition'] == new_events, dtype=np.int32)
            if not boo[i]:
                new_events[i+1:] = new_events[i:-1]
                loc.append(i)
                new_events[i] = data.iloc[i]['condition']
                boo[i] = 1 if data.iloc[i]['condition'] == new_events[i] else 0
        data.reset_index(drop=True, inplace=True)
        data_tmp = data.iloc[~data.index.isin(loc)]
    else:
        data_tmp = data

    for trial_num in range(data_tmp.shape[0]):
        if data_tmp.iloc[trial_num]['stim'] == 'car':
            events_from_annot[trial_num][2] = 16
        else:
            events_from_annot[trial_num][2] = 17

    tmin = -1
    tmax = 2.5
    baseline = -0.1
    epoch = mne.Epochs(raw, 
                    events_from_annot, 
                    event_id=stim_dict , 
                    tmin=tmin, 
                    tmax=tmax,
                    baseline = (baseline,0), 
                    preload=True, 
                    picks=['eeg'])
    # permutation
    data_n170_var = np.zeros(ntimes)
    data_cpp_var = np.zeros(ntimes)
    for i in range(ntimes):
        index_perm = np.random.permutation(data_tmp.index)
        data_perm = data_tmp.loc[index_perm].reset_index(drop=True)
        rt = data_perm['rt'].values

        chs=['PO7', 'P7', 'P8','PO8']
        time_window = np.array([[-0.18, -0.08]])+ rt.reshape(-1, 1)+ 0.2
        n_samples = int((0.1)*raw.info['sfreq'])
        epoch_slice = np.zeros((len(epoch), len(chs), n_samples))
        for j in range(len(epoch)):
            if pd.isna(data_perm['rt'][j]):
                epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]*0
            else:
                epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]

        n170_var_range = np.mean(np.mean(epoch_slice,axis=1),axis=1)
        n170_var_range = n170_var_range[np.where(n170_var_range!=0)]
        n170_var = np.var(n170_var_range)
        data_n170_var[i] = n170_var


        chs=['CPz', 'CP1', 'CP2']
        time_window = np.array([[-0.18, -0.08]])+ rt.reshape(-1, 1)+ 0.2
        epoch_slice = np.zeros((len(epoch), len(chs), n_samples))
        for j in range(len(epoch)):
            if pd.isna(data_perm['rt'][j]):
                epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]*0
            else:
                epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]

        cpp_var_range = np.mean(np.mean(epoch_slice,axis=1),axis=1)
        cpp_var_range = cpp_var_range[np.where(cpp_var_range!=0)]
        cpp_var = np.var(cpp_var_range)
        data_cpp_var[i] = cpp_var
    return  pd.DataFrame({str(subj):data_n170_var}), pd.DataFrame({str(subj):data_cpp_var})


In [24]:
n170_vars=pd.DataFrame()
cpp_vars=pd.DataFrame()
for i in range(17):
    if i+1 != 2:
        n170_var = permutate_var(subj=i+1, ntimes=500)[0]        
        cpp_var = permutate_var(subj=i+1, ntimes=500)[1]
        n170_vars = pd.concat([n170_vars,n170_var],axis=1).reset_index(drop=True)
        cpp_vars = pd.concat([cpp_vars,cpp_var],axis=1).reset_index(drop=True)
    else:
        pass
n170_vars.to_csv('results/n170_vars.csv', sep='\t', index=False)
cpp_vars.to_csv('results/cpp_vars.csv', sep='\t', index=False)

Opening raw data file preprocessed_data/sub-001_pre.fif...
    Range : 0 ... 548320 =      0.000 ...  1070.938 secs
Ready.
Reading 0 ... 548320  =      0.000 ...  1070.938 secs...


<ipython-input-22-74e201dd57c3>:4: RuntimeWarning: This filename (preprocessed_data/sub-001_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-001_pre.fif...
    Range : 0 ... 548320 =      0.000 ...  1070.938 secs
Ready.
Reading 0 ... 548320  =      0.000 ...  1070.938 secs...


<ipython-input-22-74e201dd57c3>:4: RuntimeWarning: This filename (preprocessed_data/sub-001_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-003_pre.fif...
    Range : 0 ... 566517 =      0.000 ...  1106.479 secs
Ready.
Reading 0 ... 566517  =      0.000 ...  1106.479 secs...


...

Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-017_pre.fif...
    Range : 0 ... 548300 =      0.000 ...  1070.898 secs
Ready.
Reading 0 ... 548300  =      0.000 ...  1070.898 secs...


<ipython-input-22-74e201dd57c3>:4: RuntimeWarning: This filename (preprocessed_data/sub-017_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-017_pre.fif...
    Range : 0 ... 548300 =      0.000 ...  1070.898 secs
Ready.
Reading 0 ... 548300  =      0.000 ...  1070.898 secs...


<ipython-input-22-74e201dd57c3>:4: RuntimeWarning: This filename (preprocessed_data/sub-017_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
target = np.mean(pd.read_csv('results/data_beh_eeg.csv', sep='\t')['cpp_var'])
data = np.mean(pd.read_csv('results/cpp_vars.csv', sep='\t'),axis=1)
p = sum(data < target) / len(data)